# Importing libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import xml.dom.minidom as parser
from bs4 import BeautifulSoup as bs

import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
import sklearn

# Data Processing

### Data Collection

Roll-call votes are pulled from senate.gov and associated bills are pulled from congress.gov. 

In [43]:
total_bills = []
total_votes = []
count = 0

for senate in range(101, 117):
  for session in [1,2]:

    response_0 = requests.get(f"https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_{senate}_{session}.xml")
    tree = parser.parseString(response_0.text)
    votes = tree.documentElement.getElementsByTagName("vote")

    for vote in votes:
      try:
        issue = vote.getElementsByTagName("issue")[0].childNodes[0].data.strip().split()
        vote_number = vote.getElementsByTagName("vote_number")[0].childNodes[0].data.strip()
        question = vote.getElementsByTagName("question")[0].childNodes[0].data.strip()
        if (question != "On Passage of the Bill"):
          continue
        if (issue[0] in ["S.", "H.R."]):

          response_1 = requests.get(f"https://www.senate.gov/legislative/LIS/roll_call_votes/vote{senate}{session}/vote_{senate}_{session}_{vote_number}.xml")
          tree = parser.parseString(response_1.text)
          members = tree.documentElement.getElementsByTagName("member")

          roll_call = list(filter(lambda member: member.getElementsByTagName("last_name")[0].childNodes[0].data.strip() == "Leahy", members))[0]
          if (len(roll_call.getElementsByTagName("vote_cast")) == 0):
            continue
          else:
            roll = roll_call.getElementsByTagName("vote_cast")[0].childNodes[0].data
            if (roll == 'Yea'):
              vote_b = 1
            else:
              vote_b = 0


          if (issue[0] == "S."):
            congress = "senate"
          else:
            congress = "house"

          response_2 = requests.get(f"https://www.congress.gov/bill/{senate}th-congress/{congress}-bill/{issue[1]}/text?format=txt")
          try:
            bill = bs(response_2.text).find("pre").get_text()
          except:
            continue

          count += 1
          if (count % 10 == 0):
            print(f"{senate} {session}: {count}")
          total_bills.append(bill)
          total_votes.append(vote_b)
      except:
        continue

101 1: 10
101 2: 20
102 1: 30
102 2: 40
103 1: 50
103 2: 60
104 1: 70
104 1: 80
104 1: 90
104 2: 100
104 2: 110
104 2: 120
105 2: 130
105 2: 140
105 2: 150
106 1: 160
106 1: 170
106 2: 180
106 2: 190
106 2: 200
107 1: 210
107 1: 220
107 1: 230
107 2: 240
107 2: 250
108 1: 260
108 1: 270
108 1: 280
108 2: 290
108 2: 300
109 1: 310
109 1: 320
109 2: 330
109 2: 340
109 2: 350
110 1: 360
110 1: 370
110 1: 380
110 2: 390
110 2: 400
111 1: 410
111 1: 420
111 1: 430
111 2: 440
112 1: 450
112 2: 460
112 2: 470
113 1: 480
113 2: 490
114 1: 500
114 1: 510
114 2: 520
114 2: 530
115 1: 540
116 1: 550
116 1: 560
116 2: 570


### Data Cleaning

Reformatting the XML votes of 'Yea' or 'Nay' to a binary 1 or 0, respectively

In [44]:
# reformat the roll-call votes

votes = list(map(lambda vote: vote.getElementsByTagName("vote_cast"), votes))

Y_data = list(map(lambda vote: 1 if vote == "Yea" else 0, votes))



Evaluating the Always-Yes predictive strategy

In [45]:
sum(total_votes) / len(total_votes)


0.819614711033275

Removing the zero-meaning words from all of the bills. Because we are not using sentence structure/word order, and just what words are present in the bill, this won't change the "meaning" of the bill, in the eyes of the neural network.

Our zero-meaning words are pulled from two sources. The first is the top 100 most frequent unigram (single word) in the English language. General inspection was done to decide the cut-off from the list, as the top 100 is an arbitrary choice. The second is a list of English stopwords, which are defined to be words that carry no meaning.

In [46]:
import json
# Getting negative data cleaning

top_100_unigrams = pd.read_csv('frequency/unigram_freq.csv').iloc[:101]['word'].to_list()
stopwords = json.loads(open('frequency/stop-words-english.json').read())

null_words = set(top_100_unigrams + stopwords)

for i, bill in enumerate(total_bills):
    total_bills[i] = ' '.join(filter(lambda x: x.lower() not in null_words, bill.split()))


Removing an special characters that come with text data

In [47]:
# Preprocessing
def remove_string_special_characters(s):
      
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', '', s)
    stripped = re.sub('_', '', stripped)
      
    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)
      
    # Remove start and end white spaces
    stripped = stripped.strip()
    if stripped != '':
            return stripped.lower()

bills = list(map(remove_string_special_characters, total_bills))

### Data Processing

Creating a frequency dictionary for both unigrams and bigrams for all the words in all the bills.

In [48]:
# Get the uni- and bi- gram frequencies

bigram_freq = {}
unigram_freq = {}

for bill in bills:

    for unigram in bill.split():
        unigram_freq[unigram] = unigram_freq.get(unigram,0) + 1

    for bigram in list(nltk.bigrams(bill.split())):
        bigram_freq[bigram] = bigram_freq.get(bigram,0) + 1

Sorting to get the top 4000 n-grams from all the bills.

In [49]:
# Get the top K most frequent n-gram features
K = 4000

unigrams = list({key: val for key, val in sorted(unigram_freq.items(), key = lambda ele: ele[1], reverse = True)})[0:K]
bigrams = list(map(lambda x: x[0] + ' ' + x[1], list({key: val for key, val in sorted(bigram_freq.items(), key = lambda ele: ele[1], reverse = True)})[0:K]))

In [50]:
def binary_feature_map(documents, features):
    x = []
    for doc in documents:
        x1 = []
        for feat in features:
            if (doc.find(feat) != -1):
                x1.append(1)
            else:
                x1.append(0)
        x.append(x1)
    return x

def bag_feature_map(documents, features):
    x = []
    for doc in documents:
        x1 = []
        for feat in features:
            x1.append(doc.count(feat))
        x.append(x1 / np.linalg.norm(x1))
    return x

Processing each bill's text to create 4000-dimension vectors to represent the bill. For the binary features, we are seeing whether a unigram or bigram in found in the bill or not. For the bag-of-words, we are looking at the count of that unigram or bigram in that bill.

In [51]:
# Creating the binary features for the top unigrams for each bill
X_uni_binary = binary_feature_map(bills, unigrams)

# Creating the binary features for the top bigrams for each bill
X_bi_binary = binary_feature_map(bills, bigrams)

# Creating the bag-of-words count for the top unigrams for each bill
X_uni_bag = bag_feature_map(bills, unigrams)

# Creating the bag-of-words count for the top bigrams for each bill
X_bi_bag = bag_feature_map(bills, bigrams)

In [52]:
import csv

def create_CSV(data, path):
    with open(path, 'x') as f:
        write = csv.writer(f)
        write.writerows(data)

In [53]:
data_votes = list(map(lambda x: [x], total_votes))

In [54]:
create_CSV(X_uni_binary, 'data/uni_binary_small.CSV')
create_CSV(X_uni_bag, 'data/uni_bag_small.CSV')
create_CSV(X_bi_binary, 'data/bi_binary_small.CSV')
create_CSV(X_bi_bag, 'data/bi_bag_small.CSV')

In [55]:
create_CSV(data_votes, 'data/votes_small.CSV')
create_CSV(unigrams, 'data/unigrams_small.CSV')
create_CSV(bigrams, 'data/bigrams_small.CSV')

# Building the Neural Network

In [9]:
# importing the processed data

X_uni_binary = pd.read_csv('data/uni_binary.CSV').to_numpy()
X_bi_binary = pd.read_csv('data/bi_binary.CSV').to_numpy()
X_uni_bag = pd.read_csv('data/uni_binary.CSV').to_numpy()
X_bi_bag = pd.read_csv('data/bi_binary.CSV').to_numpy()

Y_data = pd.read_csv('data/votes.CSV').to_numpy().flatten()

In [56]:
X_uni_binary_small = pd.read_csv('data/uni_binary_small.CSV').to_numpy()
X_bi_binary_small = pd.read_csv('data/bi_binary_small.CSV').to_numpy()
X_uni_bag_small = pd.read_csv('data/uni_binary_small.CSV').to_numpy()
X_bi_bag_small = pd.read_csv('data/bi_binary_small.CSV').to_numpy()

Y_data_small = pd.read_csv('data/votes_small.CSV').to_numpy().flatten()

In [57]:
def model_train_test_split(x_data, y_data):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)
    return [x_train, x_test, y_train, y_test]

In [59]:
# Unigram Binary Features
unigram_binary = model_train_test_split(X_uni_binary_small, Y_data_small)

# Bigram Binary Features
bigram_binary = model_train_test_split(X_bi_binary_small, Y_data_small)

# Unigram Bag-of-Words Features
unigram_bag = model_train_test_split(X_uni_bag_small, Y_data_small)

# Bigram Bag-of-Words Features
bigram_bag = model_train_test_split(X_bi_bag_small, Y_data_small)

In [60]:
def print_dimensions(name, data):
    print(name)
    print('X_train:',np.shape(data[0]))
    print('y_train:',np.shape(data[2]))
    print('X_test:',np.shape(data[1]))
    print('y_test:',np.shape(data[3]))
    print('\n')

In [61]:
print_dimensions('Unigram Binary', unigram_binary)
print_dimensions('Bigram Binary', bigram_binary)
print_dimensions('Unigram Bag', unigram_bag)
print_dimensions('Bigram Bag', bigram_bag)

Unigram Binary
X_train: (427, 4000)
y_train: (427,)
X_test: (143, 4000)
y_test: (143,)


Bigram Binary
X_train: (427, 4000)
y_train: (427,)
X_test: (143, 4000)
y_test: (143,)


Unigram Bag
X_train: (427, 4000)
y_train: (427,)
X_test: (143, 4000)
y_test: (143,)


Bigram Bag
X_train: (427, 4000)
y_train: (427,)
X_test: (143, 4000)
y_test: (143,)




In [62]:
import keras
from keras.models import Sequential   # importing Sequential model
from keras.layers import Dense        # importing Dense layers

In [63]:
def create_model():  
    # declaring model

    basic_model = Sequential()

    # Adding layers to the model

    # Input Layer
    basic_model.add(Dense(units = 2000, activation = 'relu', input_dim = 4000))

    # Hidden Layers
    basic_model.add(Dense(units=2000, activation='relu', input_dim=4000))

    """
    basic_model.add(Dense(units = 1000, activation = 'relu', input_dim = 2000))
    basic_model.add(Dense(units = 500, activation = 'relu', input_dim = 1000))
    basic_model.add(Dense(units=250, activation='relu', input_dim=500))
    basic_model.add(Dense(units=125, activation='relu', input_dim=250))
    basic_model.add(Dense(units = 63, activation = 'relu', input_dim = 125))
    """

    # Output Layer
    basic_model.add(Dense(1, activation = 'hard_sigmoid'))

    # compiling the model
    # opt = tf.keras.optimizers.Adam()
    opt = tf.keras.optimizers.SGD(learning_rate=0.0001)

    basic_model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

    return basic_model

In [64]:
def train_test_model(data, model, epochs):

    # training the model
    model.fit(data[0], data[2], epochs=epochs)

    # Test, Loss and accuracy
    loss_and_metrics = model.evaluate(data[1], data[3])

    print('Loss = ',loss_and_metrics[0])
    print('Accuracy = ',loss_and_metrics[1])

An Always-Yes model for the big data would give an accuracy of 0.6361980460341116
An Always-Yes model for the small data would give an accuracy of 0.819614711033275

In [65]:
train_test_model(unigram_binary, create_model(), 10)

Epoch 1/10
14/14 [==============================] - 1s 20ms/step - loss: 0.6436 - accuracy: 0.7260
Epoch 2/10
14/14 [==============================] - 0s 19ms/step - loss: 0.6040 - accuracy: 0.8009
Epoch 3/10
14/14 [==============================] - 0s 19ms/step - loss: 0.5744 - accuracy: 0.8103
Epoch 4/10
14/14 [==============================] - 0s 21ms/step - loss: 0.5510 - accuracy: 0.8126
Epoch 5/10
14/14 [==============================] - 0s 20ms/step - loss: 0.5330 - accuracy: 0.8103
Epoch 6/10
14/14 [==============================] - 0s 19ms/step - loss: 0.5200 - accuracy: 0.8126
Epoch 7/10
14/14 [==============================] - 0s 19ms/step - loss: 0.5102 - accuracy: 0.8150
Epoch 8/10
14/14 [==============================] - 0s 19ms/step - loss: 0.5037 - accuracy: 0.8150
Epoch 9/10
14/14 [==============================] - 0s 19ms/step - loss: 0.4987 - accuracy: 0.8150
Epoch 10/10
5/5 [==============================] - 0s 6ms/step - loss: 0.4798 - accuracy: 0.8392
Loss =  0.47

In [33]:
train_test_model(bigram_binary, create_model(), 10)

Epoch 1/10
142/142 [==============================] - 3s 16ms/step - loss: 0.6760 - accuracy: 0.6334
Epoch 2/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6738 - accuracy: 0.6347
Epoch 3/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6719 - accuracy: 0.6347
Epoch 4/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6702 - accuracy: 0.6347
Epoch 5/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6687 - accuracy: 0.6345
Epoch 6/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6674 - accuracy: 0.6343
Epoch 7/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6663 - accuracy: 0.6343
Epoch 8/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6653 - accuracy: 0.6343
Epoch 9/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6644 - accuracy: 0.6343
Epoch 10/10
48/48 [==============================] - 0s 6ms/step - loss: 0.6630 - accuracy:

In [35]:
train_test_model(unigram_bag, create_model(), 10)

Epoch 1/10
142/142 [==============================] - 3s 16ms/step - loss: 0.6650 - accuracy: 0.6327
Epoch 2/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6634 - accuracy: 0.6327
Epoch 3/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6623 - accuracy: 0.6327
Epoch 4/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6616 - accuracy: 0.6327
Epoch 5/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6609 - accuracy: 0.6327
Epoch 6/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6604 - accuracy: 0.6327
Epoch 7/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6599 - accuracy: 0.6327
Epoch 8/10
142/142 [==============================] - 2s 17ms/step - loss: 0.6594 - accuracy: 0.6327
Epoch 9/10
142/142 [==============================] - 2s 16ms/step - loss: 0.6589 - accuracy: 0.6327
Epoch 10/10
48/48 [==============================] - 0s 6ms/step - loss: 0.6564 - accuracy:

In [37]:
train_test_model(bigram_bag, create_model(), 15)

Epoch 1/15
142/142 [==============================] - 3s 16ms/step - loss: 0.7183 - accuracy: 0.3675
Epoch 2/15
142/142 [==============================] - 2s 16ms/step - loss: 0.7017 - accuracy: 0.4216
Epoch 3/15
142/142 [==============================] - 2s 17ms/step - loss: 0.6914 - accuracy: 0.5172
Epoch 4/15
142/142 [==============================] - 2s 16ms/step - loss: 0.6844 - accuracy: 0.5676
Epoch 5/15
142/142 [==============================] - 2s 17ms/step - loss: 0.6795 - accuracy: 0.5978
Epoch 6/15
142/142 [==============================] - 2s 16ms/step - loss: 0.6759 - accuracy: 0.6299
Epoch 7/15
142/142 [==============================] - 2s 17ms/step - loss: 0.6732 - accuracy: 0.6341
Epoch 8/15
142/142 [==============================] - 2s 16ms/step - loss: 0.6710 - accuracy: 0.6380
Epoch 9/15
142/142 [==============================] - 2s 17ms/step - loss: 0.6693 - accuracy: 0.6378
Epoch 10/15
142/142 [==============================] - 2s 17ms/step - loss: 0.6678 - accura